In [1]:
import open3d as o3d
import numpy as np
import copy
import os
import sys

sys.path.append('..')
# import open3d_tutorial as o3dut



In [14]:
def draw_registration_result(source, target, transformation):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    
    # source_temp.paint_uniform_color([1,0.706,0])
    # target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    # o3d.visualization.draw_geometries([source_temp])
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                    #   zoom=0.4459,
                                    #   front=[0.9288, -0.2951, -0.2242],
                                    #   lookat=[1.6784, 2.0612, 1.4451],
                                    #   up=[-0.3402, -0.9189, -0.1996]
                                    )

In [15]:
source = o3d.io.read_point_cloud("./4_30_Data/point_cloud/left_1.ply")
target = o3d.io.read_point_cloud("./4_30_Data/point_cloud/middle_1.ply")
threshold = 0.01
trans_init = np.asarray([[1.0, 0.011, 0.0, 0.0],
                        [-0.139, 0.967, -0.215, 0.0],
                        [0.487, 0.255, 0.835, 0.0],
                        [0.0, 0.0, 0.0, 1.0]])
draw_registration_result(source, target, trans_init)

[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [4]:
print("initial aligment")
eval = o3d.pipelines.registration.evaluate_registration(source, target, threshold, trans_init)
print(eval)

initial aligment
RegistrationResult with fitness=1.000000e+00, inlier_rmse=5.881652e-05, and correspondence_set size of 307200
Access transformation to get result.


In [ ]:
print("apply point-to-point ICP")
reg_p2p = o3d.pipelines.registration.registration_icp(source, target, threshold, trans_init, 
                                                      o3d.pipelines.registration.TransformationEstimationPointToPoint())
print(reg_p2p)
print("transformation is: ")
transformation = reg_p2p.transformation
print(transformation)
draw_registration_result(source, target, transformation)

In [ ]:
print("apply point-to-point ICP")

reg_p2p = o3d.pipelines.registration.registration_icp(source, target, threshold, trans_init, 
                                                      o3d.pipelines.registration.TransformationEstimationPointToPoint(),
                                                      o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000))
print(reg_p2p)
print("transformation is: ")
transformation = reg_p2p.transformation
print(transformation)
draw_registration_result(source, target, transformation)

: 

In [16]:
print("1. Load two point clouds and show initial pose")
# demo_colored_icp_pcds = o3d.data.DemoColoredICPPointClouds()
# source = o3d.io.read_point_cloud(demo_colored_icp_pcds.paths[0])
# target = o3d.io.read_point_cloud(demo_colored_icp_pcds.paths[1])

# draw initial alignment
# current_transformation = np.identity(4)
draw_registration_result(source, target, trans_init)

1. Load two point clouds and show initial pose
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [18]:
# point to plane ICP
# current_transformation = np.identity(4)
print("2. Point-to-plane ICP registration is applied on original point")
print("   clouds to refine the alignment. Distance threshold 0.02.")
# Estimate normals for the source point cloud
source.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

# Estimate normals for the target point cloud
target.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))

result_icp = o3d.pipelines.registration.registration_icp(
    source, target, 0.02, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPlane())
print(result_icp)
draw_registration_result(source, target,
                                        result_icp.transformation)

2. Point-to-plane ICP registration is applied on original point
   clouds to refine the alignment. Distance threshold 0.02.
RegistrationResult with fitness=1.000000e+00, inlier_rmse=3.462883e-05, and correspondence_set size of 307200
Access transformation to get result.
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [23]:
# colored pointcloud registration
# This is implementation of following paper
# J. Park, Q.-Y. Zhou, V. Koltun,
# Colored Point Cloud Registration Revisited, ICCV 2017
voxel_radius = [0.04, 0.02, 0.01]
max_iter = [50, 30, 14]
current_transformation = trans_init
print("3. Colored point cloud registration")
for scale in range(1):
    iter = max_iter[scale]
    radius = voxel_radius[scale]
    print([iter, radius, scale])

    print("3-1. Downsample with a voxel size %.2f" % radius)
    source_down = source.voxel_down_sample(radius)
    target_down = target.voxel_down_sample(radius)

    print("3-2. Estimate normal.")
    source_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))
    target_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius * 2, max_nn=30))
    
    mls_radius = 0.05
    upsampled_source = source_down.voxel_down_sample_and_trace(mls_radius, source.get_min_bound(), source.get_max_bound())[0]
    upsampled_target = source_down.voxel_down_sample_and_trace(mls_radius, target.get_min_bound(), target.get_max_bound())[0]


    print("3-3. Applying colored point cloud registration")
    result_icp = o3d.pipelines.registration.registration_colored_icp(
        upsampled_source, upsampled_target, radius, current_transformation,
        o3d.pipelines.registration.TransformationEstimationForColoredICP(),
        o3d.pipelines.registration.ICPConvergenceCriteria(relative_fitness=1e-6,
                                                          relative_rmse=1e-6,
                                                          max_iteration=iter))
    current_transformation = result_icp.transformation
    print(result_icp, current_transformation)
draw_registration_result(source, target,
                                        result_icp.transformation)

3. Colored point cloud registration
[50, 0.04, 0]
3-1. Downsample with a voxel size 0.04
3-2. Estimate normal.
3-3. Applying colored point cloud registration
RegistrationResult with fitness=1.000000e+00, inlier_rmse=3.298287e-04, and correspondence_set size of 1
Access transformation to get result. [[-1.09453699e+00 -7.96696405e-02 -2.26206333e-01 -3.80898697e-04]
 [-1.39000000e-01  9.67000000e-01 -2.15000000e-01  0.00000000e+00]
 [-1.97883269e-01 -2.42484532e-01 -8.03775898e-01 -4.25880397e-04]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[Open3D WARNING] GLFW Error: Cocoa: Failed to find service port for display


In [ ]:
[-1.39000000e-01  9.67000000e-01 -2.15000000e-01  0.00000000e+00]
 [ 5.69254327e-01  2.55023035e-01  8.32050703e-01 -9.68557265e-05]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]